In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Dec  6 12:24:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.models as models
from torch.utils.data import random_split, ConcatDataset
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import random
import argparse
from google.colab import drive
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive/MyDrive/


[Errno 2] No such file or directory: 'gdrive/MyDrive/'
/content/gdrive/My Drive/classifier


In [ ]:
cd ..

/content/gdrive/My Drive


In [ ]:
seed = 0

In [ ]:
# setting device to cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


In [ ]:
transformEdge=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.9474, 0.9474, 0.9474), (0.2038, 0.2038, 0.2038))])
transformSketch=transforms.Compose([transforms.Resize(96),transforms.ToTensor(),transforms.Normalize((0.9474, 0.9474, 0.9474), (0.2038, 0.2038, 0.2038))])

Sketch = torchvision.datasets.ImageFolder("Data/sketchy-database/Final Edgemaps/", transform=transformSketch)
Edge = torchvision.datasets.ImageFolder("Data/STL-10/STL edge maps/",transform=transformEdge)

data = ConcatDataset([Sketch, Edge])

# Calculate split lengths
total_size = len(data)
print(total_size)
train_size = round(0.9*total_size)
valid_size = round(0.05*total_size)
test_size = round(0.05*total_size)-1
print(train_size,valid_size,test_size)

train_set, valid_set, test_set = random_split(data, [train_size,valid_size,test_size], generator=torch.Generator().manual_seed(seed))


4875
4388 244 243


In [ ]:
# getting one sample image
sample_loader = DataLoader(dataset=train_set,batch_size=1, shuffle=True) 
inputs,classes = next(iter(sample_loader))

In [ ]:
print(classes)

tensor([4])


In [ ]:
print(inputs)

tensor([[[[0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581],
          [0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581],
          [0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581],
          ...,
          [0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581],
          [0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581],
          [0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581]],

         [[0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581],
          [0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581],
          [0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581],
          ...,
          [0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581],
          [0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581],
          [0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581]],

         [[0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581],
          [0.2581, 0.2581, 0.2581,  ..., 0.2581, 0.2581, 0.2581],
          [0.2581, 0.2581, 0.2581,  ..., 0

In [ ]:
# must convert to cuda.FloatTensor
inputs = inputs.type(torch.cuda.FloatTensor)

In [ ]:
cd classifier/

/content/gdrive/My Drive/classifier


In [ ]:
ls

 BestClassifier.pt               classifier_skyler.ipynb
 classifier_baseline.ipynb       FinalClassifier.ipynb
'Classifier Integration.ipynb'   TrainedClassifier.pt


In [ ]:
# defining Neural nets for classifier

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.name = 'classifier'
        self.fc1 = nn.Linear(1000, 500)
        self.fc2 = nn.Linear(500, 200)    
        self.fc3 = nn.Linear(200, 5)

    def forward(self, x):
        x = x.view(-1, 1000)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        x = self.fc3(x)
        return x

# resnet50 with trainable parameters 
resnet50T = models.resnet50(pretrained=True).to(device)
for param in resnet50T.parameters():
  param.requires_grad = False

In [ ]:
# initializing neural net 
added = Classifier().to(device)
transfer = resnet50T

#loading trained and saved classifier
classifier = torch.load("TrainedClassifier.pt")
added.load_state_dict(classifier["net"])
transfer.load_state_dict(classifier["transfer"])

<All keys matched successfully>

In [ ]:
def predict(transfer,model, img):
  img = img.to(device)
  outputs = model(transfer(inputs))
  pred = outputs.max(1, keepdim=True)[1]
  pred = pred.cpu()
  print(pred)
  return pred

In [ ]:
p = predict(transfer,added,inputs)

tensor([[4]])
